# Recognition - Image retrieval

Image retrieval, Content-Based Image Retrieval (CBIR) is a problem of the computer vision field.
The goal is to retrieve images similar to a given query image in an unlabeled (possibly large) database of images.

We will build a pipeline to perform such process.
The program will sort out the images in terms of similarity to the query.
We'll use several pethods and compare their performances.

Oxford 5k retrieval dataset: http://www.robots.ox.ac.uk/~vgg/data/oxbuildings/

credit to Filip Radenovic
https://github.com/filipradenovic/revisitop

Overview:

Using Filips code: compute performance on roxford5k based on pretrained features (python3).

Code mAP computation

Play with Keras
Use a pre-trained network
Compute global descriptors
Apply L2 normalization
Compute similarity and measure performance

More fun: 
Apply PCA to reduce the size of descriptors
Extract different layers
Extract description from different networks: inception, ResNet, DenseNet, etc.
Build a new dataset: faces, human letters, any crazy idea.


 ### Using Colab

In [ ]:
#mount your drive
from google.colab import drive
drive.mount("/content/drive") # Don't change this.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!ls drive/My\ Drive
import os
#################################################CHANGE TO YOUR PATH
os.chdir('/content/drive/MyDrive/ECM/ComputerVision/TP4_ret_archive_Etus21/python/')###################################################

!pwd



/content/drive/MyDrive/ECM/ComputerVision/TP4_ret_archive_Etus21/python


In [ ]:
!pwd
!ls


/content/drive/MyDrive/ECM/ComputerVision/TP4_ret_archive_Etus21/python
CV_TP4_retrieval21_TAZROUTE.ipynb  example_process_distractors.py
dataset.py			   example_process_images.py
dataset.pyc			   feat_vgg19_C5_avg_l2.npy
download.py			   imlist.npy
download.pyc			   __pycache__
evaluate.py			   Qfeat_vgg19_C5_avg_l2.npy
evaluate.pyc			   qimlist.npy
example_evaluate.py


# Use Filip's code

### Download the image dataset and precomputed features



In [ ]:
!mkdir /content/data
!ls /content/data

mkdir: cannot create directory ‘/content/data’: File exists


In [ ]:
from download import *


download_datasets("/content/data")###########################################
download_features("/content/data")###########################################

>> Dataset roxford5k directory does not exist. Creating: /content/data/datasets/roxford5k/jpg
>> Downloading dataset roxford5k archive oxbuild_images.tgz...
>> Extracting dataset roxford5k archive oxbuild_images.tgz...
>> Extracted, deleting dataset roxford5k archive oxbuild_images.tgz...
>> Downloading dataset roxford5k ground truth file...
>> Downloading dataset roxford5k features file roxford5k_resnet_rsfm120k_gem.mat...


In [ ]:
!ls /content/data

datasets  features


### Evaluate the downloaded features performances on roxford5k

In [ ]:

import os
import numpy as np

from scipy.io import loadmat

from dataset import configdataset
from download import download_datasets, download_features
from evaluate import compute_map

data_root = "/content/data"###########################################

# Set test dataset: roxford5k | rparis6k
test_dataset = 'roxford5k'

#---------------------------------------------------------------------
# Evaluate
#---------------------------------------------------------------------

print('>> {}: Evaluating test dataset...'.format(test_dataset)) 
# config file for the dataset
# separates query image list from database image list, when revisited protocol used
cfg = configdataset(test_dataset, os.path.join(data_root, 'datasets'))
(cfg)
# load query and database features
print('>> {}: Loading features...'.format(test_dataset))    
features = loadmat(os.path.join(data_root, 'features', '{}_resnet_rsfm120k_gem.mat'.format(test_dataset)))
Q = features['Q']
X = features['X']

# perform search
print('>> {}: Retrieval...'.format(test_dataset))
sim = np.dot(X.T, Q)
ranks = np.argsort(-sim, axis=0)

# revisited evaluation
gnd = cfg['gnd']

# search for easy
gnd_t = []
for i in range(len(gnd)):
    g = {}
    g['ok'] = np.concatenate([gnd[i]['easy']])
    g['junk'] = np.concatenate([gnd[i]['junk'], gnd[i]['hard']])
    gnd_t.append(g)
#mapE, apsE, mprE, prsE = compute_map(ranks, gnd_t, ks)
mapE, apsE, mprE, prsE = compute_map(ranks, gnd_t)

# search for easy & hard
gnd_t = []
for i in range(len(gnd)):
    g = {}
    g['ok'] = np.concatenate([gnd[i]['easy'], gnd[i]['hard']])
    g['junk'] = np.concatenate([gnd[i]['junk']])
    gnd_t.append(g)
#mapM, apsM, mprM, prsM = compute_map(ranks, gnd_t, ks)
mapM, apsM, mprM, prsM = compute_map(ranks, gnd_t)

# search for hard
gnd_t = []
for i in range(len(gnd)):
    g = {}
    g['ok'] = np.concatenate([gnd[i]['hard']])
    g['junk'] = np.concatenate([gnd[i]['junk'], gnd[i]['easy']])
    gnd_t.append(g)
#mapH, apsH, mprH, prsH = compute_map(ranks, gnd_t, ks)
mapH, apsH, mprH, prsH  = compute_map(ranks, gnd_t)

print('>> {}: mAP E: {}, M: {}, H: {}'.format(test_dataset, np.around(mapE*100, decimals=2), np.around(mapM*100, decimals=2), np.around(mapH*100, decimals=2)))
#print('>> {}: mP@k{} E: {}, M: {}, H: {}'.format(test_dataset, np.array(ks), np.around(mprE*100, decimals=2), np.around(mprM*100, decimals=2), np.around(mprH*100, decimals=2)))


#####map2, aps2 = My_compute_map(sim, gnd_t)

>> roxford5k: Evaluating test dataset...
>> roxford5k: Loading features...
>> roxford5k: Retrieval...
>> roxford5k: mAP E: 84.81, M: 64.67, H: 38.47


In [ ]:
print(ranks[0])

[2956 4290  581 3041 2293 3112 2651 3431 1258 2651 3829  933  607 3829
  607 4538  314  314 1506 1884  631  631 2868 3080 3218 2468 2468 2468
 2468 2468 4883 1242 4883 1654  972 3951 3951 3951 3951 3951 3798 1986
 1986   86 3798 3723 3723 3723 3723 3723 4300 1946 2319 2752 4706   69
  239 3907 4725 3112 3431 4586 4803  655 1767 1767  759 4659  248 2641]


In [ ]:
print(Q.shape)
print(X.shape)

print(sim.shape)
print(ranks.shape)
print(len(gnd_t))

(2048, 70)
(2048, 4993)
(4993, 70)
(4993, 70)
70


### Here is a fonction that performs L2 normalisation
Code your own one and check whether you obtain the same results



In [ ]:
import sklearn
from sklearn.preprocessing import normalize

# return descriptors that are L2 normalized
def My_norm_L2(x,axis):
    
    return(normalize(x, norm='l2', axis=axis))




# Do some coding

In [ ]:
Q = features['Q']
X = features['X']
X = My_norm_L2(X,0)
Q = My_norm_L2(Q,0)
sim = np.dot(X.T, Q)
ranks = np.argsort(-sim, axis=0)
# search for easy
gnd_t = []
for i in range(len(gnd)):
    g = {}
    g['ok'] = np.concatenate([gnd[i]['easy']])
    g['junk'] = np.concatenate([gnd[i]['junk'], gnd[i]['hard']])
    gnd_t.append(g)
    


In [ ]:
ranks.shape

(4993, 70)

### TO DO: code a fonction that compute the mAP (mean average precision)

In [ ]:
# compute mean Average Precision given the ground truth
def My_compute_map(sim, gt):
    
    return map, aps
    


In [ ]:
def computeee_ap(sim, nres):
    ranks = np.argsort(-sim, axis=0)
    nimgranks = len(ranks)

    # accumulate trapezoids in PR-plot
    ap = 0

    recall_step = 1. / nres

    for j in np.arange(nimgranks):
        rank = ranks[j]

        if rank == 0:
            precision_0 = 1.
        else:
            precision_0 = float(j) / rank

        precision_1 = float(j + 1) / (rank + 1)

        ap += (precision_0 + precision_1) * recall_step / 2.

    return ap

In [ ]:
def My_compute_map(ranks, gt):


    map = 0.
    nq = len(gt) 
    aps = np.zeros(nq) # list of average precision for each queries

    for i in range(len(gt)):
        gt_ok = np.array(gt[i]['ok'])



        gt_junk = np.array(gnd[i]['junk'])

# positions ordonées d'images positives + junk voir pos/junk
        gt_ok = np.array(gt[i]['ok'])
  # les deux boucles permettent de generer des listes pour chaque queries [T,T,F,T ....]
        pos= []
        for x in np.arange(ranks.shape[0]) : 
            if ranks[x,i] in gt_ok:
                pos.append(True)
            else : 
                pos.append(False)
        junk=[]
        for x in np.arange(ranks.shape[0]) : 
            if ranks[x,i] in gt_junk:
                junk.append(True)
            else : 
                junk.append(False)


        
        if len(junk)== 0:  # tout élément dans le ranking sera positive (sum de 1 / nb d'échantillons = 1)
            aps[i]=1
            
        else:
            k = 0;
            junk_index = 0;
            # si une image FP apparait avant une image TP =====> il faut mettre à jour les indexs :
            positive_index = 0
            while (positive_index < len(pos)): # parcours de tout élément de la liste positive
                while (junk_index < len(junk)):  # parcours de tout élément de la liste junk
                  if pos[positive_index] > junk[junk_index]: 
                      k += 1   # compteur de parcours de la liste
                      junk_index += 1   # l'é

                pos[ip] = pos[ip] - k
                ip += 1
        # Calculer l'AP pour cette query
        ap = computeee_ap(pos, gt_ok)
        
        aps[i] = ap

        map = map + ap

        pos += 1 # get it to 1-based
    map = np.mean(aps)

    return map,pos



# link : https://stackoverflow.com/questions/40457331/information-retrieval-evaluation-python-precision-recall-f-score-ap-map

#Explain the link between Average precision, precision and recall at each step.

In [ ]:
mapH  = My_compute_map(ranks, gnd_t)
mapH

# la boucle de génération de gt pour chaque query ralentit le code / on peut penser à une fonction numpy qui peut faire ça (comme filipe a utilisé dans son repo )

# Some deep learning now

Have a look at https://keras.io/ and https://keras.io/applications/
We will now work on applying a pre-trained Network to the roxford5k dataset.

## To do:
We want to build global descriptors for each image of the dataset (and for all the queries).
 - build a descriptor based on the last convolutionnal layer (perform average pooling)
 - similarly build a descriptor based on the two first fully-connected layers.
 - measure performance for each extraction method.
 
Note 1: similarity measure between queries and images of the database is performed by l2 normalizing all vectors then dot product.

Note 2: the performance will be measured for the hard version of the ground truth. You may not consider junk images in our case.

Note 3: performance wil be low (~7 to 15% map). It's normal.
 
 

In [ ]:
datapath = '../data/datasets/roxford5k/jpg/'####################################################

network = VGG19(weights='imagenet')#, input_shape=(fscale, fscale, 3))
network = Model(inputs=network.input, outputs=network.get_layer('fc2').output)#Use only the pre-trained convolutional layers
network.trainable = Falsedatapath = '../data/datasets/roxford5k/jpg/'####################################################


qname = np.load('qimlist.npy')

fname = np.load('imlist.npy')

all_qfeat = np.zeros(shape=(len(qname),4096), dtype=np.float32)
#im_batch_Q = np.zeros(shape=(len(qname),224,224,3), dtype=np.float32)

all_feat = np.zeros(shape=(len(fname),4096), dtype=np.float32)
#im_batch = np.zeros(shape=(len(fname),224,224,3), dtype=np.float32)

cpt = 0
for i in range(len(qname)):
    img_path = os.path.join(datapath,qname[i]+'.jpg')

    t = time.time()

    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    #im_batch_Q[cpt,:,:,:] = x

    qfeatures = network.predict(x)
    #qfeatures = qfeatures.flatten()

    all_qfeat[cpt,:]= qfeatures
    cpt = cpt+1
    elapsed = time.time() - t
    print(elapsed)
    print(cpt)

cpt =0
for i in range(len(fname)):
    img_path = os.path.join(datapath,fname[i]+'.jpg')

    t = time.time()

    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    #im_batch[cpt,:,:,:] = x

    features = network.predict(x)

    all_feat[cpt,:]= features
    cpt = cpt+1
    elapsed = time.time() - t
    print(elapsed)
    print(cpt)

np.save('all_feat_fc2.npy',all_feat)
np.save('all_qfeat_fc2.npy',all_qfeat)
#    all_feat = np.load('PATH_TO_MY_FILE.npy')

In [ ]:
# Tests can be performed here
print(all_qfeat.shape)
print(all_feat.shape)
print(qfeatures.shape)
all_qfeat_norm = My_norm_L2(all_qfeat,axis=1)
all_feat_norm = My_norm_L2(all_feat,axis=1)
print(np.sum(all_feat_norm**2.0, axis=1))


In [ ]:
sim = np.dot(all_feat_norm, all_qfeat_norm.T)
ranks = np.argsort(-sim, axis=0)
ranks.shape

In [1]:
#getting ground truth / hard way


gnd_t = []
for i in range(len(gnd)):
    g = {}
    g['ok'] = np.concatenate([gnd[i]['easy']])
    g['junk'] = np.concatenate([gnd[i]['junk'], gnd[i]['hard']])
    gnd_t.append(g)
    

In [ ]:
mapH  = compute_map(ranks, gnd_t)
mapH

In [ ]:
# j'ai pas su comment manipuler ceci / est ce ce sont des features extraites d'un vgg d'une couche autre que de laquelle j'ai coupé le réseau
import numpy as np
queries_c5 = np.load('Qfeat_vgg19_C5_avg_l2.npy')
print(aa.shape)
all_c5 = np.load('feat_vgg19_C5_avg_l2.npy')
print(cc.shape)
